In [1]:
import pandas as pd
import numpy as np

import re
from tqdm import tqdm



# Processing

In [2]:
df = pd.read_csv('data/main_df.csv')
df.head()

,Name,Surah,Ayat,Arabic,Translation - Muhammad Tahir-ul-Qadri,Translation - Arthur J,Translation - Marmaduke Pickthall,Tafaseer - Tafsir al-Jalalayn,Tafaseer - Tanwir al-Miqbas min Tafsir Ibn Abbas,EnglishTitle,ArabicTitle,RomanTitle,NumberOfVerses,NumberOfRukus,PlaceOfRevelation
0,The Opening,1,1,بِسمِ ٱلله الرَّحْمٰنِ الرَّحِيـمِ,"All praise be to Allah alone, the Sustainer of...","In the Name of God, the Merciful, the Compassi...","In the name of Allah, the Beneficent, the Merc...",In the Name of God the Compassionate the Merciful,"In the name of Allah, the Beneficent, the Merc...",Al-Fatihah,ٱلْفَاتِحَة,al-Ḥamd,7,1,Makkah
1,The Opening,1,2,ٱلْحَمْدُ للَّهِ رَبِّ ٱلْعَالَمِينَ,"Most Compassionate, Ever-Merciful,","Praise belongs to God, the Lord of all Being,","Praise be to Allah, Lord of the Worlds,",In the Name of God the name of a thing is that...,And on his authority it is related that Ibn 'A...,Al-Fatihah,ٱلْفَاتِحَة,al-Ḥamd,7,1,Makkah
2,The Opening,1,3,ٱلرَّحْمـٰنِ ٱلرَّحِيمِ,Master of the Day of Judgment.,"the All-merciful, the All-compassionate,","The Beneficent, the Merciful.",The Compassionate the Merciful that is to say ...,(The Beneficent) the Gentle. (The Merciful) th...,Al-Fatihah,ٱلْفَاتِحَة,al-Ḥamd,7,1,Makkah
3,The Opening,1,4,مَـٰلِكِ يَوْمِ ٱلدِّينِ,(O Allah!) You alone do we worship and to You ...,the Master of the Day of Doom.,"Owner of the Day of Judgment,",Master of the Day of Judgement that is the day...,(Owner of the Day of Judgement) the Arbitrator...,Al-Fatihah,ٱلْفَاتِحَة,al-Ḥamd,7,1,Makkah
4,The Opening,1,5,إِيَّاكَ نَعْبُدُ وَإِيَّاكَ نَسْتَعِينُ,"Show us the straight path,",Thee only we serve; to Thee alone we pray for ...,Thee (alone) we worship; Thee (alone) we ask f...,You alone we worship and You alone we ask for ...,"(Thee (alone) we worship), we turn to you as t...",Al-Fatihah,ٱلْفَاتِحَة,al-Ḥamd,7,1,Makkah


In [3]:
translation_col = list(df.columns)
translation_col = [x for x in translation_col if "Translation" in x]

tafaseer_col = list(df.columns)
tafaseer_col = [x for x in tafaseer_col if "Tafaseer" in x]

In [4]:
arabic = []
text=[]
for index, row in df.iterrows():
    arabic.append(row['Arabic'])
    t = ""
    t += row['Name'] + " | " + str(row['Arabic'])+" | "+ str(row['Surah'])+" | "+str(row['Ayat'])+" | "
    t += row['EnglishTitle'] + " | " + str(row['ArabicTitle'])+" | "+ str(row['RomanTitle'])+" | "
    t += row['PlaceOfRevelation'] + " | "
    for j in translation_col:
        t += row[j] + " + "
    t += " | "
    for j in tafaseer_col:
        t+= row[j] + " + "
    t = t[:-3]
    text.append(t)

In [5]:
text[0]

'The Opening | بِسمِ ٱلله الرَّحْمٰنِ الرَّحِيـمِ | 1 | 1 | Al-Fatihah | ٱلْفَاتِحَة | al-Ḥamd | Makkah | All praise be to Allah alone, the Sustainer of all the worlds, + In the Name of God, the Merciful, the Compassionate + In the name of Allah, the Beneficent, the Merciful. +  | In the Name of God the Compassionate the Merciful + In the name of Allah, the Beneficent, the Merciful.'

In [6]:
arabic[0]

'بِسمِ ٱلله الرَّحْمٰنِ الرَّحِيـمِ'

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Preprocess the dataset
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(text)

# Define the search function
def search(query, top_k=5):
    query_vec = vectorizer.transform([query])
    sim_scores = cosine_similarity(query_vec, X).flatten()
    top_indices = sim_scores.argsort()[::-1][:top_k]
    top_paragraphs = [text[i] for i in top_indices]
    
    
    
    return top_paragraphs

In [8]:
x = search("inheritance")

In [9]:
x[0].split(" | ")

['Smoke',
 'كَذَلِكَ وَأَوْرَثْنَاهَا قَوْماً آخَرِينَ',
 '44',
 '28',
 'Ad-Dukhaan',
 'ٱلدُّخَان',
 'ad-Dukhān',
 'Makkah',
 'So it happened! We made another people the inheritors of all that. + Even so; and We bequeathed them upon another people. + Even so (it was), and We made it an inheritance for other folk; + ',
 'So it was kadhālika is the predicate of a subject that is missing such as ‘the matter was so’ and We made these an inheritance that is to say their riches for another people namely the Children of Israel. + (Even so (it was)) We destroyed them, (and We made it an inheritance for other folk) We made the Children of Israel inherit all that after they were destroyed;']

In [10]:
x[0].split(" | ")[-1].split(" + ")

['So it was kadhālika is the predicate of a subject that is missing such as ‘the matter was so’ and We made these an inheritance that is to say their riches for another people namely the Children of Israel.',
 '(Even so (it was)) We destroyed them, (and We made it an inheritance for other folk) We made the Children of Israel inherit all that after they were destroyed;']

In [11]:
x[0].split(" | ")[-2].split(" + ")

['So it happened! We made another people the inheritors of all that.',
 'Even so; and We bequeathed them upon another people.',
 'Even so (it was), and We made it an inheritance for other folk;',
 '']

In [12]:
from deep_translator import GoogleTranslator
# translate Urdu text to English


In [13]:


urdu_text = "ہم پاکستانی"
english_text = GoogleTranslator(source='auto', target='en').translate(urdu_text)
print(english_text)  # output: "We are Pakistani"

# translate Roman text to English
roman_text = "mera naam Rashid hai"
english_text = GoogleTranslator(source='auto', target='en').translate(roman_text)
print(english_text)  # output: "My name is Rashid"

We are Pakistanis
my name is rashid
